# **Final change detection**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Empty the folder

In [26]:
import shutil
import os
shutil.rmtree('/content/drive/MyDrive/capstone/P_png')
os.mkdir('/content/drive/MyDrive/capstone/P_png')
#shutil.rmtree('/content/drive/MyDrive/capstone/P')
#os.mkdir('/content/drive/MyDrive/capstone/P')
shutil.rmtree('/content/drive/MyDrive/capstone/P cut')
os.mkdir('/content/drive/MyDrive/capstone/P cut')
shutil.rmtree('/content/drive/MyDrive/capstone/diff')
os.mkdir('/content/drive/MyDrive/capstone/diff')
shutil.rmtree('/content/drive/MyDrive/capstone/changemap')
os.mkdir('/content/drive/MyDrive/capstone/changemap')
shutil.rmtree('/content/drive/MyDrive/capstone/cleanchangemap')
os.mkdir('/content/drive/MyDrive/capstone/cleanchangemap')

# Sentinel-2 data collection and cropping

If you want to use Sentinel 2 data you can follow these steps to modify the path, if you already have an image to detect you can skip this section

## **1. Sentinel Data Download**

1) ESA website: https://scihub.copernicus.eu/dhus/#/home

2) Register an account or log in

3) Select target region and data type (S2A_MSIL1C)

4) Add data to the cart

5) Download the .meta4 file and save it to the local PC

In [ ]:
from xml.dom.minidom import parse
from data_downloader import downloader

In [ ]:
folder_out = r'D:\sentinal2_data'  # A file that store output data
url_file = r'D:\sentinal2_data\meta\products.meta4'  # Downloaded the products.meta4 

data = parse(url_file).documentElement
urls = [i.childNodes[0].nodeValue for i in data.getElementsByTagName('url')]

downloader.download_datas(urls,folder_out)

## **2. Data Processing: atmospheric correction**

1) Download Sen2Cor software : http://step.esa.int/main/third-party-plugins-2/sen2cor/

2) Use command prompt to switch the path to the S2A_MSIL1C data

3) Enter L2A_Process file name --resolution=10 --refresh command to process S2A_MSIL1C data and convert it to S2A_MSIL2A data

## **3. Data format conversion**
Save sentinal data in Geotif format for subsequent processing

In [ ]:
from osgeo import gdal
import os
import numpy as np
from osgeo import gdal, osr, ogr
import glob
os.environ['CPL_ZIP_ENCODING'] = 'UTF-8'

In [ ]:
def S2tif(filename):

    print(filename)
    root_ds = gdal.Open(filename)    # Open the raster dataset
    print(type(root_ds))
    """
    The return result is a list. Each element in the list is a tuple. Each tuple contains a description of the path 
    to the dataset, metadata, and so on. The first element in the tuple describes the full path of the data subset.
    """
    ds_list = root_ds.GetSubDatasets()  # Get the subdata set. 
    visual_ds = gdal.Open(ds_list[0][0])  # Open the path to the first data subset
    visual_arr = visual_ds.ReadAsArray()  # Read the data in the dataset as ndarray
    print(visual_arr.shape)
    
#     visual_ds = gdal.Open(ds_list[1][0])  #The ds_list has 4 subsets, the first part is the path and the rest part is the data information
#     visual_arr = visual_ds.ReadAsArray()   
#     visual_ds = gdal.Open(ds_list[2][0])  
#     visual_arr = visual_ds.ReadAsArray()
#     visual_ds = gdal.Open(ds_list[3][0])  
#     visual_arr = visual_ds.ReadAsArray()


    # Create Tif file
    band_count = visual_ds.RasterCount # Band number
    print(band_count)
    xsize = visual_ds.RasterXSize
    ysize = visual_ds.RasterYSize
    out_tif_name = filename.split(".SAFE")[0] + ".tif"
    driver = gdal.GetDriverByName("GTiff")
    out_tif = driver.Create(out_tif_name, xsize, ysize, band_count, gdal.GDT_Float32)
    out_tif.SetProjection(visual_ds.GetProjection())  # Set projection coordinates
    out_tif.SetGeoTransform(visual_ds.GetGeoTransform())

    for index, band in enumerate(visual_arr):
        band = np.array([band])
        for i in range(len(band[:])):
            out_tif.GetRasterBand(index + 1).WriteArray(band[i]) # Writeout the data
    out_tif.FlushCache()  # Synchronizes data to disks
    out_tif = None  # Close the TIF file  

In [ ]:
if __name__ == "__main__":
    from osgeo import gdal
    SAFE_Path = (r'D:\sentinal2_data\L2A') # Tif file storage path
    data_list = glob.glob(SAFE_Path + "\\*.SAFE")

    #filename = ('E:\\RSDATA\\Sentinel2\\L2A\\S2A_MSIL2A_20210220T024731_N9999_R132_T51STA_20210306T024402.SAFE\\MTD_MSIL2A.xml')
    for i in range(len(data_list)):
        data_path = data_list[i]
        filename = data_path + "\\MTD_MSIL2A.xml"
        S2tif(filename)
        print(data_path + "-----Turn tif successfully ")
    print("----Conversion end----")

# Cropping and formatting of images

In [27]:
import cv2 as cv
import os
from PIL import Image

def Convert_To_Png_AndCut(dir):
    files = os.listdir(dir)
    ResultPath1 = "/content/drive/MyDrive/capstone/P_png/" # Define the path to save the format after conversion
    ResultPath2 = "/content/drive/MyDrive/capstone/P cut/" # Define the save path after cropping
    file_name = []
    
    for file in files:
        a, b = os.path.splitext(file) # File name of the split image map
        file_name.append(a)
        this_dir = os.path.join(dir + file) # Build Save Path + Filename
     
        img = cv.imread(this_dir, 1) # Read tif images
     
        cv.imwrite(ResultPath1 + a + "_" + ".png", img) # Save as png
     
        hight = img.shape[0] #Get width and height
        width = img.shape[1]
        #Defining the cut size
        w = 650 # Width
        h = 650 # Height
        _id = 1 # Crop result save file name: 0 - N in ascending order
        i = 0
        row = 0
        column = 0 
        while (i + h <= hight): # Control the height and leave out the excess fixed size sum of the image
            j = 0
            row = row + 1
            while (j + w <= width):  # Control the width and leave the excess fixed size sum of the image alone
                column = column + 1
                cropped = img[i:i + h, j:j + w] # Cropped coordinates are [y0:y1, x0:x1]
                cv.imwrite(ResultPath2 + a + "_" + str(_id) + b, cropped)
                _id += 1
                j += w
            i = i + h
    column = column / row
    return file_name[0], file_name[1], _id - 1, row, int(column)


In [28]:
if __name__ == '__main__':
    _path = "/content/drive/MyDrive/capstone/P/"  # Remote sensing tiff image location path
    # Crop image map
    after, before, number, row, column = Convert_To_Png_AndCut(_path)

# Change detection for unsupervised learning methods

In [14]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter
import scipy
from skimage.transform import resize
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## **1. Difference image and the Eigen vector space**

In this method,a non-overlapping blocks are taken of size 5 x 5 from the difference image and flatten them into row vectors. The image can be resized to make both the dimensions a multiple of 5 by scipy.misc.imresize(). Collection of these row vectors forms a vector set. In change_detection.py script, find_vector_set() does exactly this. If the size of our difference image is m x n, then the number of rows in the vector set would be {m x n}/{5 x 5} .

In [15]:
def find_vector_set(diff_image, new_size):
   
    i = 0
    j = 0
    vector_set = np.zeros((int(new_size[0] * new_size[1] / 25), 25))

    print('\nvector_set shape',vector_set.shape)
    
    while i < vector_set.shape[0]:
        while j < new_size[0]:
            k = 0
            while k < new_size[1]:
                block   = diff_image[j:j+5, k:k+5]
                #print(i,j,k,block.shape)
                feature = block.ravel()
                vector_set[i, :] = feature
                k = k + 5
            j = j + 5
        i = i + 1
        
            
    mean_vec   = np.mean(vector_set, axis = 0)    
    vector_set = vector_set - mean_vec
    
    return vector_set, mean_vec

## **2. Building the feature vector space**

Building the FVS involves again taking 5 x 5 blocks from the difference image, flattening them, and lastly projecting them onto the EVS, only this time, the blocks will be overlapping. A vector space (VS) is first made by constructing one vector for each pixel of the difference image such a way that one 5 x 5 block is actually a pixel’s 5 x 5 neighborhood. It is to be noted here that by this logic, 4 boundary rows and 4 boundary columns pixels won’t get any feature vectors since they won’t have a 5 x 5 neighborhood. (We can manage with this exclusion of these pixels, since it is safe to assume here that any changes occurring would be concentrated in the middle regions of the images, rather than the edges). So, we will have (m x n)- 8 feature vectors in the FVS, all 25 dimensional. Projecting the FVS to the 25 dimensional EVS simply means to perform the following matrix multiplication

(VS)((m x n - 8) x 25) .(EVS)(25 x 25) = (FVS)(m x n - 8) x 25

Function find_FVS() determines the feature vector space for us. The function is similar to find_vector_set(), but extracts overlapping blocks from the difference image.

In [16]:
def find_FVS(EVS, diff_image, mean_vec, new):
    
    i = 2 
    feature_vector_set = []
    
    while i < new[0] - 2:
        j = 2
        while j < new[1] - 2:
            block = diff_image[i-2:i+3, j-2:j+3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j+1
        i = i+1
        
    FVS = np.dot(feature_vector_set, EVS)
    FVS = FVS - mean_vec
    print("\nfeature vector space size",FVS.shape)
    return FVS

## **3. Clustering of the feature vector space, and change map**

The feature vectors for the pixels carry information whether the pixels have characteristics of a changed pixel or an unchanged one. Having constructed the feature vector space, we now need to cluster it so that the pixels can be grouped into two disjoint classes. K-means algorithm is used to do that. Thus each pixel will get assigned to a cluster in such a way that the distance between the cluster’s mean vector and the pixel’s feature vector is the least. Each pixel gets a label from 1 to K, which denotes the cluster number that they belong to.

In [17]:
def clustering(FVS, components, new):
    
    kmeans = KMeans(components, verbose = 0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count  = Counter(output)

    least_index = min(count, key = count.get)            
    print(new[0],new[1])
    change_map  = np.reshape(output,(new[0] - 4, new[1] - 4))
    
    return least_index, change_map

During this experiments, it was empirically found that the best results were obtained with K = 3. Thus the argument components in clustering() will be 3. Remember, even though we have to do divide the pixels into 2 categories, we have chosen K = 3, instead of 2. Now how do we decide which of these clusters contains the pixels that belong to the changed class? It can be postulated that the cluster which contains the lowest number of pixels (denoted by variable least_index) is the cluster denoting the changed class, since the background remains more or less the same in satellite images and the changes occurred are comparatively less. Also, the mean of this cluster will be the highest. The reason behind the highest value of mean for that cluster is that the values of the difference image pixels in a region where some changes have occurred are higher than the values of pixels in the regions where there is no change.

Thus, in conclusion, the cluster with the lowest number of pixels, and also the highest mean is the cluster belonging to the changed class.
With this information, we will now build a change map – a binary image to show the output of change detection. We have chosen to keep the background black and will show the changes in white, i.e., intensity value of those pixels will be 255. You can do the reverse as well.

In [18]:
def find_PCAKmeans(imagepath1, imagepath2, after, i):
    
    print('Operating')
    
    image1 = cv2.imread(imagepath1)
    image2 = cv2.imread(imagepath2)
    image1=image1[:,:,0]
    image2=image2[:,:,0]
    print(image1.shape,image2.shape) 
    new_size = np.asarray(image1.shape) / 5
    new_size = new_size.astype(int) * 5
    image1 = resize(image1, (new_size))
    image2 = resize(image2, (new_size))
    diff_image = abs(image1 - image2)   
    imageio.imwrite('/content/drive/MyDrive/capstone/diff/' + after + '_' + str(i) + '.png', diff_image)
    print('\nBoth images resized to ',new_size)
        
    vector_set, mean_vec = find_vector_set(diff_image, new_size)
    pca     = PCA()
    pca.fit(vector_set)
    EVS = pca.components_   
    FVS     = find_FVS(EVS, diff_image, mean_vec, new_size)
    print('\ncomputing k means')
    
    components = 3
    least_index, change_map = clustering(FVS, components, new_size)
    change_map[change_map == least_index] = 255 ## change map formula
    change_map[change_map != 255] = 0   
    change_map = change_map.astype(np.uint8)
    kernel     = np.asarray(((0,0,1,0,0),
                             (0,1,1,1,0),
                             (1,1,1,1,1),
                             (0,1,1,1,0),
                             (0,0,1,0,0)), dtype=np.uint8)
    cleanChangeMap = cv2.erode(change_map,kernel)
    imageio.imwrite('/content/drive/MyDrive/capstone/changemap/' + after + '_' + str(i) + '.png', change_map)
    imageio.imwrite('/content/drive/MyDrive/capstone/cleanchangemap/' + after + '_' + str(i) + '.png', cleanChangeMap)

In [29]:
path = "/content/drive/MyDrive/capstone/P cut/"
for i in range(1, number + 1):
    a= path + before + '_' + str(i) + '.png'
    b= path + after + '_' + str(i) + '.png'
    find_PCAKmeans(a, b, after, i) 

Operating
(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)

feature vector space size (417316, 25)

computing k means
650 650
Operating


(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)

feature vector space size (417316, 25)

computing k means


650 650
Operating
(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var



feature vector space size (417316, 25)

computing k means
650 650
Operating


...

650 650
Operating
(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)

feature vector space size (417316, 25)

computing k means
650 650


Operating
(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var



feature vector space size (417316, 25)

computing k means
650 650


Operating
(650, 650) (650, 650)

Both images resized to  [650 650]

vector_set shape (16900, 25)

feature vector space size (417316, 25)

computing k means
650 650


# Image stitching and multiply

Stitching the images in the diff folder

In [30]:
import PIL.Image as Image
import os

IMAGES_PATH = '/content/drive/MyDrive/capstone/diff/'
IMAGES_FORMAT = ['.jpg', '.JPG','.jpeg', '.png']

#row = 7
#column = 10


IMAGE_SIZE = 650# Size of each small image

IMAGE_ROW = row  # Image spacing, i.e. how many rows there are when combined into one image
IMAGE_COLUMN = column  # Image spacing, i.e. how many columns there are when combined into one image
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/stich result/'  # Address after image conversion
 
# Get the names of all the images under the address of the image set
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

x=image_names
a='.'
b='_'
image_names.sort(key = lambda x:int(x.split(a)[0].split(b)[-2])) # Image sorting
print(len(image_names))


 
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) # Create a new image
    # Iterate through the loop, pasting each image into the corresponding position in order
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    to_image.save(IMAGE_SAVE_PATH + 'diff_merged.png') # Save new image
image_compose() # Calling functions

70


Stitching the images in the changemap folder

In [31]:
import PIL.Image as Image
import os

IMAGES_PATH = '/content/drive/MyDrive/capstone/changemap/'
IMAGES_FORMAT = ['.jpg', '.JPG','.jpeg', '.png']

#row = 7
#column = 10


IMAGE_SIZE = 650# Size of each small image

IMAGE_ROW = row  # Image spacing, i.e. how many rows there are when combined into one image
IMAGE_COLUMN = column  # Image spacing, i.e. how many columns there are when combined into one image
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/stich result/'  # Address after image conversion
 
# Get the names of all the images under the address of the image set
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

x=image_names
a='.'
b='_'
image_names.sort(key = lambda x:int(x.split(a)[0].split(b)[-2])) # Image sorting
print(len(image_names))


 
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) # Create a new image
    # Iterate through the loop, pasting each image into the corresponding position in order
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    to_image.save(IMAGE_SAVE_PATH + 'changemap_merged.png') # Save new image
image_compose() # Calling functions

70


Stitching the images in the cleanchangemap folder

In [32]:
import PIL.Image as Image
import os

IMAGES_PATH = '/content/drive/MyDrive/capstone/cleanchangemap/'
IMAGES_FORMAT = ['.jpg', '.JPG','.jpeg', '.png']

#row = 7
#column = 10


IMAGE_SIZE = 650# Size of each small image

IMAGE_ROW = row  # Image spacing, i.e. how many rows there are when combined into one image
IMAGE_COLUMN = column  # Image spacing, i.e. how many columns there are when combined into one image
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/stich result/'  # Address after image conversion
 
# Get the names of all the images under the address of the image set
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

x=image_names
a='.'
b='_'
image_names.sort(key = lambda x:int(x.split(a)[0].split(b)[-2])) # Image sorting
print(len(image_names))


 
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) # Create a new image
    # Iterate through the loop, pasting each image into the corresponding position in order
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    to_image.save(IMAGE_SAVE_PATH + 'cleanchangemap_merged.png') # Save new image
image_compose() # Calling functions

70


Stitching images from the image label folder

In [ ]:
import PIL.Image as Image
import os

IMAGES_PATH = '/content/drive/MyDrive/capstone/image label/'
IMAGES_FORMAT = ['.jpg', '.JPG','.jpeg', '.png']

#row = 21
#column = 22


IMAGE_SIZE = 650# Size of each small image

IMAGE_ROW = row  # Image spacing, i.e. how many rows there are when combined into one image
IMAGE_COLUMN = column  # Image spacing, i.e. how many columns there are when combined into one image
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/stich result/'  # Address after image conversion
 
# Get the names of all the images under the address of the image set
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

x=image_names
a='.'
b='_'
image_names.sort(key = lambda x:int(x.split('.')[0]))
print(len(image_names))


 
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) # Create a new diagram
    # Iterate through the loop, pasting each image into the corresponding position in order
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    to_image.save(IMAGE_SAVE_PATH + 'labeled_merged.png') # Save new image
image_compose() # Calling functions

462


Multiplying diff images and labelled images

In [ ]:
# Importing Image and ImageChops module from PIL package
from PIL import Image, ImageChops
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None	
# create label
label = Image.open(r"/content/drive/MyDrive/capstone/stich result/diff_merged.png")

# ValueError: images do not match, need to match image mode
label = label.convert("RGB")

# creat mask
mask = Image.open(r"/content/drive/MyDrive/capstone/stich result/labeled_merged.png")
mask = mask.convert("RGB")	

# applying multiply method
im3 = ImageChops.multiply(label,mask)
im3.save('/content/drive/MyDrive/capstone/stich result/multiply_diff.png')

Multiplying changemap images and labelled images

In [ ]:
# Importing Image and ImageChops module from PIL package
from PIL import Image, ImageChops
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None	
# create label
label = Image.open(r"/content/drive/MyDrive/capstone/stich result/changemap_merged.png")

# ValueError: images do not match, need to match image mode
label = label.convert("RGB")

# creat mask
mask = Image.open(r"/content/drive/MyDrive/capstone/stich result/labeled_merged.png")
mask = mask.convert("RGB")	

# applying multiply method
im3 = ImageChops.multiply(label,mask)
im3.save('/content/drive/MyDrive/capstone/stich result/multiply_changemap.png')

Multiplying cleanchangmap images and labelled images

In [ ]:
# Importing Image and ImageChops module from PIL package
from PIL import Image, ImageChops
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None	
# create label
label = Image.open(r"/content/drive/MyDrive/capstone/stich result/cleanchangemap_merged.png")

# ValueError: images do not match, need to match image mode
label = label.convert("RGB")

# creat mask
mask = Image.open(r"/content/drive/MyDrive/capstone/stich result/labeled_merged.png")
mask = mask.convert("RGB")	

# applying multiply method
im3 = ImageChops.multiply(label,mask)
im3.save('/content/drive/MyDrive/capstone/stich result/multiply_cleanchangemap.png')

In [ ]:
import PIL.Image as Image
import os

IMAGES_PATH = '/content/drive/MyDrive/capstone/report/cleanchangemap/'
IMAGES_FORMAT = ['.jpg', '.JPG','.jpeg', '.png']

row = 5
column = 7


IMAGE_SIZE = 650# Size of each small image

IMAGE_ROW = row  # Image spacing, i.e. how many rows there are when combined into one image
IMAGE_COLUMN = column  # Image spacing, i.e. how many columns there are when combined into one image
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/stich result/'  # Address after image conversion
 
# Get the names of all the images under the address of the image set
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

x=image_names
a='.'
b='_'
image_names.sort(key = lambda x:int(x.split(a)[0].split(b)[-2])) # Image sorting
print(len(image_names))


 
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) # Create a new image
    # Iterate through the loop, pasting each image into the corresponding position in order
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    to_image.save(IMAGE_SAVE_PATH + 'report_cleanchangemap.png') # Save new image
image_compose() # Calling functions

35


In [ ]:
# Importing Image and ImageChops module from PIL package
from PIL import Image, ImageChops
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None	
# create label
label = Image.open(r"/content/drive/MyDrive/capstone/stich result/report_cleanchangemap.png")

# ValueError: images do not match, need to match image mode
label = label.convert("RGB")

# creat mask
mask = Image.open(r"/content/drive/MyDrive/capstone/Unet++/report/label/compose_image_label.png")
mask = mask.convert("RGB")	

# applying multiply method
im3 = ImageChops.multiply(label,mask)
im3.save('/content/drive/MyDrive/capstone/stich result/multiply_report_cleanchangemap.png')

# **Unet++ method**

Run train.py to generate an evaluation image to update the model, if you want to use it directly you can ignore this step and use the trained model for change detection

In [ ]:
!python /content/drive/MyDrive/Lamboise-Master/train.py

INFO: Using cuda
INFO: Generating dataset ...
INFO: Batch size: 1
INFO: Dataset generated
INFO: Network creation:

INFO: Starting training : 
                Type : light Unet++
                Epochs: 99
                Batch size: 1
                Data Augmentation: 2
                Learning rate: 0.001
                Device: cuda
                Reloading model : False
                Saving model : False
/content/drive/MyDrive/Lamboise-Master/train.py:84: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for epochs in range(0, num_epochs):
INFO: Epoch 0.0
Epoch 0.0: 44img [00:16,  2.67img/s, loss=0.665]
Validation: 10img [00:01,  5.09img/s, loss=tensor(7.4145, device='cuda:0')]
INFO: Train loss 0.6616742041977969
INFO: Test loss  0.831049382686615
INFO: Epoch 1.0
Epoch 1.0: 44img [00:16,  2.63img/s, loss=0.661]
Validat

**Run predict.py to do the architecture change detection (UNet++)** <br>
*!python path0 -i path1 -o path2* <br>
(whole path is needed on colab

*   path0 refers to the path of predict.py 
*   path1 refers to the path of the input file 
*   path2 refers to the path of the output file
*   files needed to be arranged as follow(under the file):
   *  instance_1
      *  before.png
      *  after.png
      *  predicted.png (result 1
   *  instance_2
      *  before.png
      *  after.png
      *  predicted.png (result 2


In [ ]:
!python /content/drive/MyDrive/capstone/Lamboise-Master/predict.py -i /content/drive/MyDrive/capstone/Unet++/change_detection -o /content/drive/MyDrive/capstone/Unet++/change_detection

Stitch label

In [ ]:
import PIL.Image as Image
import os
 
IMAGES_PATH = '/content/drive/MyDrive/capstone/Unet++/report/2018/' 
IMAGES_FORMAT = ['.png', '.PNG'] 
w=650            
h=650           
IMAGE_ROW = 5  
IMAGE_COLUMN = 7  
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/Unet++/report/2018/compose_image_2018.png'  
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

image_names.sort(key=lambda x: int(x.split(".")[0]))
if len(image_names) != IMAGE_ROW * IMAGE_COLUMN:
    raise ValueError("Doesn't match！")
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN *w, IMAGE_ROW *h)) 
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (w, h),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * w, (y - 1) * h))
    return to_image.save(IMAGE_SAVE_PATH) 
image_compose() 

Stitch 2021

In [ ]:
import PIL.Image as Image
import os
 
IMAGES_PATH = '/content/drive/MyDrive/capstone/Unet++/report/2021/' 
IMAGES_FORMAT = ['.png', '.PNG'] 
w=650            
h=650           
IMAGE_ROW = 5  
IMAGE_COLUMN = 7  
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/Unet++/report/2021/compose_image_2021.png'  
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

image_names.sort(key=lambda x: int(x.split(".")[0]))
if len(image_names) != IMAGE_ROW * IMAGE_COLUMN:
    raise ValueError("Doesn't match！")
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN *w, IMAGE_ROW *h)) 
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (w, h),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * w, (y - 1) * h))
    return to_image.save(IMAGE_SAVE_PATH) 
image_compose() 

Stitch mask

In [ ]:
import PIL.Image as Image
import os
 
IMAGES_PATH = '/content/drive/MyDrive/capstone/Unet++/report/mask/' 
IMAGES_FORMAT = ['.png', '.PNG'] 
w=650            
h=650           
IMAGE_ROW = 5  
IMAGE_COLUMN = 7  
IMAGE_SAVE_PATH = '/content/drive/MyDrive/capstone/Unet++/report/mask/compose_image_mask.png'  
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

image_names.sort(key=lambda x: int(x.split(".")[0]))
if len(image_names) != IMAGE_ROW * IMAGE_COLUMN:
    raise ValueError("Doesn't match！")
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN *w, IMAGE_ROW *h)) 
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (w, h),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * w, (y - 1) * h))
    return to_image.save(IMAGE_SAVE_PATH) 
image_compose() 

Code for predicted image processing that make the result more distingishable, including **enhance contrast & brightness**:

In [ ]:
import cv2
import numpy as np
import os
from PIL import Image, ImageEnhance

If first processing the cropped images then stitching the images:

In [ ]:
IMAGES_PATH = '/content/drive/MyDrive/capstone/Lamboise-Master/TEST' # input path（predict
IMAGES_FORMAT = ['.png', '.PNG']  

# two steps
IMAGE_SAVE_PATH_1 = '/content/drive/MyDrive/capstone/Lamboise-Master/TEST'  # saving path 1(for enhancing contrast)
IMAGE_SAVE_PATH_2 = '/content/drive/MyDrive/capstone/Lamboise-Master/TEST'  # saving path 2(for enhancing brightness)

image_names_1 = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

for image_1 in image_names_1: # increase contrast
  im = Image.open(IMAGES_PATH+'/'+image_1) # read the image
  enhancer = ImageEnhance.Contrast(im) # image contrast enhancer
  factor = 3 #increase contrast
  im_output = enhancer.enhance(factor)
  im_output.save(IMAGE_SAVE_PATH_1+'/'+image_1)

image_names_2 = [name for name in os.listdir(IMAGES_SAVE_PATH_1) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

for image_2 in image_names_2: # increase brightness
  im = Image.open(IMAGE_SAVE_PATH_1+'/'+image_2) # read the image
  enhancer = ImageEnhance.Brightness(im) # image brightness enhancer
  factor = 3 #increase brightness
  im_output = enhancer.enhance(factor)
  im_output.save(IMAGE_SAVE_PATH_2+'/'+image_2)

If first stitching then process the images **(recommended)**:


In [ ]:
#read the image
im = Image.open("/content/drive/MyDrive/capstone/Unet++/report/mask/compose_image_mask.png") # input path

#image contrast enhancer
enhancer = ImageEnhance.Contrast(im)

factor = 3 #increase contrast
im_output = enhancer.enhance(factor)
im_output.save('/content/drive/MyDrive/capstone/Unet++/report/mask/more-contrast-image.png')

im1 = Image.open("/content/drive/MyDrive/capstone/Unet++/report/mask/more-contrast-image.png")

#image brightness enhancer
enhancer = ImageEnhance.Brightness(im1)

factor = 3 #brightens the image
im_output = enhancer.enhance(factor)
im_output.save('/content/drive/MyDrive/capstone/Unet++/report/mask/brightened-image.png')

Do a **multiply** on the mask and labelled image to keep the changed zones while shows the development of the two types of architectures. 


In [ ]:
# Importing Image and ImageChops module from PIL package
from PIL import Image, ImageChops
	
# create label
label = Image.open(r"/content/drive/MyDrive/capstone/Unet++/report/label/compose_image_label.png")

# ValueError: images do not match, need to match image mode
label = label.convert("RGB")

# creat mask
mask = Image.open(r"//content/drive/MyDrive/capstone/Unet++/report/mask/brightened-image.png")
mask = mask.convert("RGB")	

# applying multiply method
im3 = ImageChops.multiply(label,mask)
im3.save('/content/drive/MyDrive/capstone/Unet++/report/result.png')